In [ ]:
from flask import Flask, request, jsonify
import pickle
import numpy as np
import pandas as pd
import json
import math

def sanitize(data):
    if isinstance(data, dict):
        return {k: sanitize(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [sanitize(v) for v in data]
    elif isinstance(data, float) and math.isnan(data):
        return None
    return data

with open("symptom_index.json") as f:
    symptoms = json.load(f)

# Load the model
model = pickle.load(open('ExtraTrees.pkl', 'rb'))

diseases = [
    '(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne', 'Alcoholic hepatitis', 'Allergy', 
    'Arthritis', 'Bronchial Asthma', 'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis', 
    'Common Cold', 'Dengue', 'Diabetes', 'Dimorphic hemmorhoids(piles)', 'Drug Reaction', 
    'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack', 'Hepatitis B', 'Hepatitis C', 
    'Hepatitis D', 'Hepatitis E', 'Hypertension', 'Hyperthyroidism', 'Hypoglycemia', 'Hypothyroidism', 
    'Impetigo', 'Jaundice', 'Malaria', 'Migraine', 'Osteoarthristis', 'Paralysis (brain hemorrhage)', 
    'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis', 'Typhoid', 
    'Urinary tract infection', 'Varicose veins', 'hepatitis A'
]


print(len(symptoms))
desc=pd.read_csv("symptoms-datasets/symptom_Description.csv")
prec=pd.read_csv("symptoms-datasets/symptom_precaution.csv")
app = Flask(__name__)


@app.route('/', methods=["GET"])
def home():
    return app.send_static_file('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    features=pd.Series([0]*131, index=symptoms)
    print(len(features))
    # Parse the JSON input
    data = request.get_json(force=True)
    print("Received symptoms:", data)

    # Set features to 1 where symptoms are present
    for symptom in data:
        if symptom in features.index:
            features[symptom] = 1

    # Convert to NumPy and reshape
    features = features.to_numpy().reshape(1, -1)
    print("Feature vector:", features.tolist())

    # Predict probabilities
    proba = model.predict_proba(features)

    # Top 5 predicted classes and their probabilities
    top5_idx = np.argsort(proba[0])[-5:][::-1]
    top5_proba = np.sort(proba[0])[-5:][::-1]
    top5_diseases = [diseases[i] for i in top5_idx]

    # Build response
    response = []
    for i in range(5):
        disease = top5_diseases[i]
        probability = float(top5_proba[i])
        
        # Disease description
        disp = desc[desc['Disease'] == disease].values[0][1] if disease in desc["Disease"].unique() else "No description available"
        
        # Precautions
        precautions = []
        if disease in prec["Disease"].unique():
            c = np.where(prec['Disease'] == disease)[0][0]
            for j in range(1, len(prec.iloc[c])):
                precautions.append(prec.iloc[c, j])
        print(disease, probability, disp, precautions)
        # Add to response
        response.append({
            'disease': disease,
            'probability': probability,
            'description': disp,
            'precautions': precautions
        })
    clean_result = sanitize(response)
    return jsonify(clean_result)


if __name__ == '__main__':
    app.run(port=5050, debug=True, use_reloader=False)


131
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5050
Press CTRL+C to quit
127.0.0.1 - - [13/May/2025 14:26:14] "POST /predict HTTP/1.1" 200 -


131
Received symptoms: {'dizziness': 1, 'throat_irritation': 1, 'chills': 1, 'cough': 1}
Feature vector: [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Allergy 0.23 An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens. ['apply calamine', 'cover area with bandage', nan, 'use ice to compress itching']
Cervical spondylosis 0.17 Cervical spondylosis is a general term for age-related wear and tear affecting the spinal disks in your neck. As the disks dehydrate and sh

127.0.0.1 - - [13/May/2025 14:26:20] "POST /predict HTTP/1.1" 200 -


131
Received symptoms: {'dizziness': 1, 'throat_irritation': 1, 'chills': 1, 'cough': 1}
Feature vector: [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Allergy 0.23 An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens. ['apply calamine', 'cover area with bandage', nan, 'use ice to compress itching']
Cervical spondylosis 0.17 Cervical spondylosis is a general term for age-related wear and tear affecting the spinal disks in your neck. As the disks dehydrate and sh